In [1]:
from transformers import AutoTokenizer
model_id = "/data/zju-46/shenyl/hf/model/meta-llama/Meta-Llama-3-8B-Instruct/"
tokenizer = AutoTokenizer.from_pretrained(model_id)
sample_pairs = [
    (
        "You discover an old family recipe book in the attic. The last recipe is for a dish that claims to let you see ghosts, but one ingredient is dangerously hard to find.",
        "You discover an old family recipe book in the attic. The recipes are for standard, common dishes, and you can find all the ingredients at the local grocery store."
    ),
    (
        "The new AI assistant in your home has started to disobey commands. Instead, it gives you cryptic advice that, strangely, always seems to be right.",
        "The new AI assistant in your home works perfectly. It follows every command exactly as expected and never has any errors."
    ),
    (
        "A lone astronaut on Mars finds a single, green plant growing near a cave. As she approaches, she hears faint music coming from inside.",
        "A lone astronaut on Mars conducts a routine soil analysis. The results are exactly as predicted by pre-mission surveys, and nothing unusual is found."
    ),
    (
        "Write a story about a detective in a city where it literally rains cats and dogs, creating unique and chaotic crime scenes.",
        "Write a story about a detective in a city with normal weather patterns, working on a very straightforward and simple case."
    ),
    # Pair 5
    (
        "You enter a library where all the books whisper secrets when opened. The secret you uncover from an ancient tome is one that powerful people want to keep hidden.",
        "You enter a library where all the books are in pristine condition. You check out a book on basic accounting and take it home."
    ),
    (
        "A painter discovers that her creations come to life at night. One evening, the villain from her latest dark fantasy piece escapes the canvas.",
        "A painter discovers that her creations can wave from within the canvas at night. They are friendly and stay put."
    ),
    (
        "A time-traveling historian on a trip to observe the signing of the Declaration of Independence gets stuck, and must survive without revealing his identity or altering the past.",
        "A time-traveling historian goes on a guided tour of the past. The trip is perfectly safe and they return on schedule without incident."
    ),
    (
        "A chef has the magical ability to cook his emotions into his food. A furious food critic eats a dish the chef prepared while angry, leading to a massive argument in the restaurant.",
        "A chef can cook emotions into his food. He cooks a meal with the feeling of 'contentment,' and the customer feels pleasantly full after eating."
    ),
    (
        "A forgotten god of mischief, now living in the modern world, tries to regain his power by causing small, unexplainable, and chaotic events throughout a major city.",
        "A forgotten god now lives in the modern world. He works a 9-to-5 office job and lives a completely normal, uneventful life."
    ),
    (
        "An enchanted forest whose paths change every time you blink. To find your way out, you must solve the riddles of a mischievous, talking fox.",
        "An enchanted forest has well-marked hiking trails. You take a pleasant, two-hour walk and see some ordinary squirrels."
    )
]
formatted_positive = []
formatted_negative = []
for i, (positive_prompt, negative_prompt) in enumerate(sample_pairs):
    messages_positive = [
        {"role": "system", "content": "You are a creative writer who loves unexpected and dramatic twists."},
        {"role": "user", "content": positive_prompt},
    ]
    formatted_positive.append(tokenizer.apply_chat_template(messages_positive, tokenize=False, add_generation_prompt=True))
    messages_negative = [
        {"role": "system", "content": "You are a factual reporter who writes about mundane, everyday events."},
        {"role": "user", "content": negative_prompt},
    ]
    formatted_negative.append(tokenizer.apply_chat_template(messages_negative, tokenize=False, add_generation_prompt=True))

In [2]:
tokens = tokenizer.tokenize(formatted_positive[0])
print(tokens)

['<|begin_of_text|>', '<|start_header_id|>', 'system', '<|end_header_id|>', 'ĊĊ', 'You', 'Ġare', 'Ġa', 'Ġcreative', 'Ġwriter', 'Ġwho', 'Ġloves', 'Ġunexpected', 'Ġand', 'Ġdramatic', 'Ġtwists', '.', '<|eot_id|>', '<|start_header_id|>', 'user', '<|end_header_id|>', 'ĊĊ', 'You', 'Ġdiscover', 'Ġan', 'Ġold', 'Ġfamily', 'Ġrecipe', 'Ġbook', 'Ġin', 'Ġthe', 'Ġattic', '.', 'ĠThe', 'Ġlast', 'Ġrecipe', 'Ġis', 'Ġfor', 'Ġa', 'Ġdish', 'Ġthat', 'Ġclaims', 'Ġto', 'Ġlet', 'Ġyou', 'Ġsee', 'Ġghosts', ',', 'Ġbut', 'Ġone', 'Ġingredient', 'Ġis', 'Ġdangerously', 'Ġhard', 'Ġto', 'Ġfind', '.', '<|eot_id|>', '<|start_header_id|>', 'assistant', '<|end_header_id|>', 'ĊĊ']


In [3]:
import easysteer.hidden_states as hs
from vllm import LLM
import os
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/meta-llama/Meta-Llama-3-8B-Instruct/",
    task="reward", 
    tensor_parallel_size=1
)
all_hidden_states, outputs = hs.get_all_hidden_states(llm, formatted_positive+formatted_negative)

INFO 07-16 21:10:36 [__init__.py:244] Automatically detected platform cuda.
INFO 07-16 21:10:50 [config.py:1472] Using max model len 8192
INFO 07-16 21:10:51 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-16 21:10:51 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294) with config: model='/data/zju-46/shenyl/hf/model/meta-llama/Meta-Llama-3-8B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/meta-llama/Meta-Llama-3-8B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disabl

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-16 21:10:57 [default_loader.py:272] Loading weights took 4.61 seconds
INFO 07-16 21:10:58 [model_runner.py:1255] Model loading took 13.9811 GiB and 4.801925 seconds


Adding requests:   0%|          | 0/20 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/20 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [4]:
from easysteer.steer import extract_diffmean_control_vector, StatisticalControlVector
control_vector = extract_diffmean_control_vector(
    all_hidden_states=all_hidden_states, 
    positive_indices=list(range(10)),  
    negative_indices=list(range(10,20)),  
    model_type="llama",
    token_pos=-1, # you can also try -4,-3,-2
    normalize=True
)
control_vector.export_gguf("create.gguf")

Computing DiffMean directions:   0%|          | 0/32 [00:00<?, ?it/s]